<a href="https://colab.research.google.com/github/kunalsonalkar/training-llms/blob/main/bytePairEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.7 MB/s eta 0:00:00


In [2]:
from importlib.metadata import version
import tiktoken
print("tiktoken version: ", version("tiktoken"))

tiktoken version:  0.7.0


In [3]:
tokenizer = tiktoken.get_encoding("gpt2")

In [4]:
tokenizer

<Encoding 'gpt2'>

In [5]:
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

In [6]:
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


In [9]:
# Sliding window approach for data masking
with open("/content/the-verdict.txt","r",encoding="utf-8") as f:
  raw_text = f.read()
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [10]:
# take the first 50 samples
enc_sample = enc_text[50:]

In [11]:
# Decide a context size and then shift input-target pairs
context_size=4
x=enc_sample[:context_size]
y=enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y: {y}")

x: [290, 4920, 2241, 287]
y: [4920, 2241, 287, 257]


In [12]:
for i in range(1, context_size+1):
  print(i)
  context = enc_sample[:i]
  desired = enc_sample[i]
  print(context, "---->", desired)

1
[290] ----> 4920
2
[290, 4920] ----> 2241
3
[290, 4920, 2241] ----> 287
4
[290, 4920, 2241, 287] ----> 257


In [13]:
# Printing the actual text
for i in range(1, context_size+1):
  print(i)
  context = enc_sample[:i]
  print(f"The context is:  {context}")
  desired = enc_sample[i]
  print(f"The desired part is: {desired}")
  print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

1
The context is:  [290]
The desired part is: 4920
 and ---->  established
2
The context is:  [290, 4920]
The desired part is: 2241
 and established ---->  himself
3
The context is:  [290, 4920, 2241]
The desired part is: 287
 and established himself ---->  in
4
The context is:  [290, 4920, 2241, 287]
The desired part is: 257
 and established himself in ---->  a


In [14]:
# Efficient data loader for getting inputs and target as PyTorch tensors
import torch
from torch.utils.data import Dataset, DataLoader
class GPTDatasetV1(Dataset):
  def __init__(self, txt, tokenizer, max_length, stride):
    self.tokenizer = tokenizer
    self.input_ids = []
    self.target_ids = []

    token_ids = tokenizer.encode(txt)

    for i in range(0, len(token_ids) - max_length, stride):
      input_chunk = token_ids[i:i+max_length]
      target_chunk = token_ids[i+1:i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]

In [15]:
# Dataloader to generate batches with input target pairs
def create_dataloader_v1(txt, batch_size=4,
                         max_length=256, stride=128, shuffle=True, drop_last=True):
  tokenizer = tiktoken.get_encoding("gpt2")
  dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
  dataloader = DataLoader(
      dataset, batch_size = batch_size, shuffle=shuffle, drop_last=drop_last)
  return dataloader

In [16]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [17]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [18]:
# batch_size > 1
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(inputs)
print("-------")
print(targets)

tensor([[  198,   198,  1544,  6204],
        [  198,  6653,  6563,  2951],
        [ 4976,     6,   582,    11],
        [31640,    12,    67, 20811],
        [ 2583,   408,    25,   366],
        [ 3022,   706,   339,   373],
        [  284,   366, 26282,   683],
        [  887,   484,  4054,   502]])
-------
tensor([[  198,  1544,  6204,  2045],
        [ 6653,  6563,  2951,  6348],
        [    6,   582,    11,   290],
        [   12,    67, 20811,     1],
        [  408,    25,   366,    40],
        [  706,   339,   373,  2636],
        [  366, 26282,   683,   510],
        [  484,  4054,   502,    11]])


In [23]:
# Token-id to Vector embedding
input_ids = torch.tensor([2,3,5,0])
vocab_size = 6
output_dim = 3

In [20]:
# setting up the manual seed for reproducing
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [24]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [25]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.3374, -0.1778, -0.1690]], grad_fn=<EmbeddingBackward0>)


In [26]:
# Extending to larger embeddings
output_dim = 256
vocab_size = 50257
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [29]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size = 8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(inputs)
print("next")
print(inputs.shape)

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
next
torch.Size([8, 4])


In [30]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [31]:
token_embeddings

tensor([[[ 0.4913,  1.1239,  1.4588,  ..., -0.3995, -1.8735, -0.1445],
         [ 0.4481,  0.2536, -0.2655,  ...,  0.4997, -1.1991, -1.1844],
         [-0.2507, -0.0546,  0.6687,  ...,  0.9618,  2.3737, -0.0528],
         [ 0.9457,  0.8657,  1.6191,  ..., -0.4544, -0.7460,  0.3483]],

        [[ 1.5460,  1.7368, -0.7848,  ..., -0.1004,  0.8584, -0.3421],
         [-1.8622, -0.1914, -0.3812,  ...,  1.1220, -0.3496,  0.6091],
         [ 1.9847, -0.6483, -0.1415,  ..., -0.3841, -0.9355,  1.4478],
         [ 0.9647,  1.2974, -1.6207,  ...,  1.1463,  1.5797,  0.3969]],

        [[-0.7713,  0.6572,  0.1663,  ..., -0.8044,  0.0542,  0.7426],
         [ 0.8046,  0.5047,  1.2922,  ...,  1.4648,  0.4097,  0.3205],
         [ 0.0795, -1.7636,  0.5750,  ...,  2.1823,  1.8231, -0.3635],
         [ 0.4267, -0.0647,  0.5686,  ..., -0.5209,  1.3065,  0.8473]],

        ...,

        [[-1.6156,  0.9610, -2.6437,  ..., -0.9645,  1.0888,  1.6383],
         [-0.3985, -0.9235, -1.3163,  ..., -1.1582, -1.13

In [32]:
token_embedding_layer

Embedding(50257, 256)

In [34]:
# get absolute positional embeddings (same as GPT based models)
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [35]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


In [37]:
pos_embeddings

tensor([[-0.8290,  0.0647,  0.1265,  ...,  0.3610,  1.0378,  0.7732],
        [-0.3407, -0.0392,  1.8435,  ...,  0.5663,  1.3551, -1.9100],
        [-1.0489, -0.4816, -0.8424,  ...,  0.9912, -0.6396,  1.3296],
        [ 0.2580,  0.3851,  0.0491,  ...,  1.3488, -1.6770, -0.4796]],
       grad_fn=<EmbeddingBackward0>)